In [2]:
import kiutils.symbol
import random
import os
import glob
import pandas as pd

#### Function for generating a unique BR ID ####
def generate_BRID(existing_BRIDs):

    # Set number of digits in unique BRID
    num_digits = 6      

    # Increment by 1 until the BRID doesn't exist
    for n in range(10**num_digits):

        # Convert to a zero padded string (to fill the desired number of digits -- e.g, 1 --> '0001')
        BR_num = str(n).zfill(num_digits)
        BRID = 'BRE-' + BR_num

        # If the new BRID isn't currently used, we've found the one!
        if BRID not in existing_BRIDs:
            break

    return BRID

In [12]:
parts_df = pd.DataFrame(columns=['ID','Name','Description','Value','Symbol','Footprint','Datasheet','Manufacturer','MPN', 'Category'])
vendors_df = pd.DataFrame(columns=['ID','Supplier','SPN','Stock'])

ID_list = []
parts_list = []
vendors_list = []

# Reading symbol libraries from our BR symbols folder
SYMBOLS_PATH = "C:/Users/JacobBrotmanKrass/Documents/GitHub/br-kicad-lib/Symbols"
os.chdir(SYMBOLS_PATH)
for lib_file in glob.glob("*.kicad_sym"):

    lib_nickname = lib_file.replace(".kicad_sym", "")
    if lib_nickname == "BR~Deprecated":
        continue
    print(lib_nickname)
    lib_path = os.path.join(SYMBOLS_PATH, lib_file)
    lib_obj = kiutils.symbol.SymbolLib()
    sym_lib = lib_obj.from_file(lib_path)
    # The Category is the library nickname, without the BR_ at the beginning
    category = lib_nickname[3:]



    # For each symbol in a given library, populate a new row in the Parts table of the database
    for symbol in sym_lib.symbols:
        symbol_path = f"{lib_nickname}:{symbol.entryName}"
        BR_ID = generate_BRID(ID_list)
        ID_list.append(BR_ID)
        properties = {property.key: property.value for property in symbol.properties}
        bad_char = b'\xc3\x82'.decode()
        print(properties)
        properties["Description"] = properties["Description"].replace(bad_char, '')
        if "Manufacturer" in properties:
            manufacturer = properties["Manufacturer"]
            mpn = properties["Manufacturer Part Num"]
        else:
            manufacturer = "None"
            mpn = "None"

        parts_list.append({"ID":BR_ID, "Name":symbol.libId, "Description":properties["Description"], "Value":properties["Value"], "Symbol":symbol_path, "Footprint":properties["Footprint"],  "Datasheet":properties["Datasheet"], "Manufacturer":manufacturer, "MPN":mpn, "Category":category})

        print(properties["Description"])

        supplier_properties = {property: properties[property] for property in properties if property[:8]=="Supplier"}
        supplier_numbers = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]=='P'}
        supplier_names = {supp_prop: supplier_properties[supp_prop] for supp_prop in supplier_properties if supp_prop[9]!='P'}

        null_strings = ["", " ", "-", "--", "~", "NA", "N/A"]
        for name in supplier_names:
            for number in supplier_numbers:
                if name[-1] == number[-1]:
                    if supplier_numbers[number] not in null_strings:
                        vendors_list.append({"ID":BR_ID, "Supplier":supplier_names[name], "SPN":supplier_numbers[number], "Stock":0})


parts_df = pd.DataFrame(parts_list)
vendors_df = pd.DataFrame(vendors_list)
                

BR_Capacitors_0201
{'Reference': 'C', 'Value': '100nF 25V', 'Footprint': 'BR_Passives:C_0201_0603Metric-minimized', 'Datasheet': 'https://www.lcsc.com/datasheet/lcsc_datasheet_2108070630_Murata-Electronics-GRM033R61E104KE14J_C2649540.pdf', 'Description': '100nF Â±10% 25V X7R 0201 Ceramic Capacitor', 'Manufacturer': 'Murata Electronics', 'Manufacturer Part Num': 'GRM033R61E104KE14J', 'Supplier 1': 'DigiKey', 'Supplier Part Num 1': '490-14571-1-ND', 'Supplier 2': 'Mouser', 'Supplier Part Num 2': '81-GRM033R61E104KE4J', 'Supplier 3': 'JLCPCB', 'Supplier Part Num 3': 'C2649540', 'ki_keywords': 'capacitor cap', 'ki_fp_filters': 'C_*'}
100nF ±10% 25V X7R 0201 Ceramic Capacitor
{'Reference': 'C', 'Value': '10nF 25V', 'Footprint': 'BR_Passives:C_0201_0603Metric-minimized', 'Datasheet': 'https://search.murata.co.jp/Ceramy/image/img/A01X/G101/ENG/GRM033R71E103KE14-01.pdf', 'Description': '10nF Â±10% 25V X7R 0201 Ceramic Capacitor', 'Manufacturer': 'Murata Electronics', 'Manufacturer Part Num': '

In [14]:
vendors_df

,ID,Supplier,SPN,Stock
0,BRE-000000,DigiKey,490-14571-1-ND,0
1,BRE-000000,Mouser,81-GRM033R61E104KE4J,0
2,BRE-000000,JLCPCB,C2649540,0
3,BRE-000001,DigiKey,490-14454-2-ND,0
4,BRE-000001,Mouser,81-GRM0335C1E12FA1D,0
...,...,...,...,...
1000,BRE-000445,JLCPCB,C9900082548,0
1001,BRE-000446,JLCPCB,C2916352,0
1002,BRE-000447,JLCPCB,C2916351,0
1003,BRE-000471,DigiKey,36-5017CT-ND,0
